# Compare two ROI files and return the score and the ROIs of False Positives and False Negatives detection

Compare two ROI .zip files, one containing manual annotation of cell event, and the one containing the DeXtrusion detection, and print the accuracy scores. It can also generate ROI .zip file containing the False Positives and False Negatives (of DeXtrusion results compared to manual one) files to allow to check them.

In [ ]:
# default parameters
talkative = True  ## print info messages
import os
from glob import glob
import numpy as np
imname = os.getcwd()
# DeXNet path, to read the configuration file
modeldir = "./deXNets/notum_all/notumAll0/"
from dextrusion.DeXtrusion import DeXtrusion
dexter = DeXtrusion(verbose=talkative)

In [ ]:
#### Parameters to choose

dxy = 15          ## distance to ground-truth ROI to consider same
dt = 4            ## distance in time to ground-truth ROI to consider same
cat = 1           ## event to score (here cell_death)
catnames = ["", "_cell_death.zip", "_cell_sop.zip", "_cell_division.zip"]

## path the ROIs files are
test_path = "../../data/notum_mix/test/"
## path the results folder where DeXtrusion ROI files are
res = test_path+"results/"

# parameters used to create the ROIs (to keep in memory in the score datasheet)
volume_thres = 800
prob_thres = 180

In [ ]:
## parameter to consider ROIs as the same within DeXtrusion (spatial and temporal distance threshold)
distxy=10
dist=4

### Write results to file
resultfile = res+"/score.csv"
header = 'Name;VolumeThreshold;ProbaThreshold;distanceXY;distanceT;TP;FP;FN;Precision;Recall' 

## Test all the files present in the result folder
resfiles = glob(res+"/*"+catnames[cat], recursive = False)
resarray = None

for resfile in resfiles:
    testname = os.path.basename(resfile)
    ind = testname.find(catnames[cat])
    testname = testname[0:ind]
    print(testname)
    paras = np.array([testname, volume_thres, prob_thres, dxy, dt])
    outfile = res+testname+catnames[cat]
    #dexter.get_rois_of_scaled_image( res+testname+catnames[cat]+"_proba.tif", volume_thres, prob_thres, distxy, dist, cat=cat, outfile=outfile)
    
    # compare the ROIs and print the scores
    score = dexter.compare_rois(cat=cat, gtroisfile=test_path+testname+catnames[cat], resroisfile=outfile, distance_xy=dxy, distance_t=dt)
    
    # write in .zip files the false detections (put # in the beginning of the lines to not do this step)
    dexter.write_falsepositives(cat=cat, resfile=res+testname+catnames[cat], gtfile=test_path+testname+catnames[cat], distance_xy=dxy, distance_t=dt)
    dexter.write_falsenegatives(cat=cat, resfile=res+testname+catnames[cat], gtfile=test_path+testname+catnames[cat],distance_xy=dxy, distance_t=dt)
    
    array = np.atleast_2d(np.append(paras, np.array(score)))
    if resarray is None:
        resarray = array
    else:
        resarray = np.append(resarray, array, axis=0)

## save the scores in a tabular file
with open(resultfile,'w') as csvfile:
    np.savetxt(csvfile, resarray, delimiter=';', header=header, fmt='%s', comments='')